# Aquatic-based atmopsheric correction for Sentinel 2 image collection for GEOAQUAWATCH

Created By: Benjamin Page (benjaminpage8@gmail.com)    

Translated to Python: Kelly Luis (kelly.m.luis@jpl.nasa.gov)    

Please reference: Page, B.P., Olmanson, L.G. and Mishra, D.R., 2019. A harmonized image processing workflow using Sentinel-2/MSI and Landsat-8/OLI for mapping water clarity in optically variable lake systems. Remote Sensing of Environment, 231, p.111284.    

Before you start, you will need to expand your notebook rate limit. Use the code below when you open up jupyter notebook in the terminal:
jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


In [ ]:
# Import Packages and Authenticate
import ee
import geemap
import folium
import geehydro
ee.Authenticate()
ee.Initialize()

In [ ]:
# S2 image search time constraint
startMonth = 1
endMonth = 12
startYear = 2022
endYear = 2022

# Or

# Target specific time frame
# Start date
startDate = '2019-06-01'

# End date
endDate = '2019-09-30'

# Cloud Filter
cloudPerc = 1

# Target image index (from console)
target_image_number = 0

tileNum = '14SQB'

lat = 33.37109359117551
lon = -97.06406547865038

In [ ]:
# Import Collections w/ Sentinel-2
MSI = ee.ImageCollection('COPERNICUS/S2')

ozone = ee.ImageCollection('TOMS/MERGED')

JRC = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')

In [ ]:
# Process 
mask = JRC.select('occurrence').gt(0)
FC_S2A = MSI.filterMetadata('MGRS_TILE', 'equals', tileNum).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloudPerc).filter(ee.Filter.calendarRange(startMonth, endMonth, 'month')).filter(ee.Filter.calendarRange(startYear, endYear, 'year')).filterMetadata('SPACECRAFT_NAME', 'equals', 'Sentinel-2A').sort('system:time_start')

In [ ]:
FC_S2A_List = FC_S2A.toList(100000)
FC_S2B = MSI.filterMetadata('MGRS_TILE', 'equals', tileNum).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloudPerc).filter(ee.Filter.calendarRange(startMonth, endMonth, 'month')).filter(ee.Filter.calendarRange(startYear, endYear, 'year')).filterMetadata('SPACECRAFT_NAME', 'equals', 'Sentinel-2B').sort('system:time_start')
FC_S2B_List = FC_S2B.toList(100000)

# merge s2a and s2b image collections
FC_combined = FC_S2A.merge(FC_S2B).sort('system:time_start')
FCList = FC_combined.toList(100000)
target_image = ee.Image(FCList.get(target_image_number))
outputGeometry = target_image.select('B1').geometry()

In [ ]:
# MAIN Atmospheric Correction
pi = ee.Image(3.141592);


def MAIN_S2A(img):
    
    # msi bands
    bands = ['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'];

    #rescale
    rescale = img.select(bands).divide(10000)
    
    # tile footprint
    footprint = rescale.geometry()
    
    # DEM 
    dem = ee.Image('USGS/SRTMGL1_003').clip(footprint)

    # ozone
    DU = ee.Image(300); # ozone @ sea level

    # Julian Day
    imgDate = ee.Date(img.get('system:time_start'))
    FOY = ee.Date.fromYMD(imgDate.get('year'),1,1)
    JD = imgDate.difference(FOY,'day').int().add(1)

    # Earth-Sun distance
    myCos = ((ee.Image(0.0172).multiply(ee.Image(JD).subtract(ee.Image(2)))).cos()).pow(2)
    cosd = myCos.multiply(pi.divide(ee.Image(180))).cos()
    d = ee.Image(1).subtract(ee.Image(0.01673)).multiply(cosd).clip(footprint)

    # sun azimuth
    SunAz = ee.Image.constant(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).clip(footprint)

    # sun zenith
    SunZe = ee.Image.constant(img.get('MEAN_SOLAR_ZENITH_ANGLE')).clip(footprint)
    cosdSunZe = SunZe.multiply(pi.divide(ee.Image(180))).cos() # in degrees
    sindSunZe = SunZe.multiply(pi.divide(ee.Image(180))).sin() # in degrees

    # sat zenith
    SatZe = ee.Image.constant(img.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5')).clip(footprint)
    cosdSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).cos()
    sindSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).sin()
  
    # sat azimuth
    SatAz = ee.Image.constant(img.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5')).clip(footprint)

    # relative azimuth
    RelAz = SatAz.subtract(SunAz)
    cosdRelAz = RelAz.multiply(pi.divide(ee.Image(180))).cos()

    # Pressure
    P = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(dem)).pow(5.25588)).multiply(0.01)
    Po = ee.Image(1013.25)
    
    ESUN = ee.Image(ee.Array([ee.Image(img.get('SOLAR_IRRADIANCE_B1')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B2')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B3')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B4')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B5')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B6')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B7')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B8')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B8A')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B11')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B12'))]
                  )).toArray().toArray(1)

    ESUN = ESUN.multiply(ee.Image(1))
    
    ESUNImg = ESUN.arrayProject([0]).arrayFlatten([bands])
    
    # create empty array for the images
    imgArr = rescale.select(bands).toArray().toArray(1)
    
    
    # pTOA to Ltoa
    Ltoa = imgArr.multiply(ESUN).multiply(cosdSunZe).divide(pi.multiply(d.pow(2)))

    # band centers
    bandCenter = ee.Image(444).divide(1000).addBands(ee.Image(496).divide(1000)).addBands(ee.Image(560).divide(1000)).addBands(ee.Image(664).divide(1000)).addBands(ee.Image(704).divide(1000)).addBands(ee.Image(740).divide(1000)).addBands(ee.Image(782).divide(1000)).addBands(ee.Image(835).divide(1000)).addBands(ee.Image(865).divide(1000)).addBands(ee.Image(1613).divide(1000)).addBands(ee.Image(2202).divide(1000)).toArray().toArray(1)

    # ozone coefficients
    koz = ee.Image(0.0040).addBands(ee.Image(0.0244)).addBands(ee.Image(0.1052)).addBands(ee.Image(0.0516)).addBands(ee.Image(0.0208)).addBands(ee.Image(0.0112)).addBands(ee.Image(0.0079)).addBands(ee.Image(0.0021)).addBands(ee.Image(0.0019))                          .addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1)
                        
    # Calculate ozone optical thickness
    Toz = koz.multiply(DU).divide(ee.Image(1000))

    # Calculate TOA radiance in the absense of ozone
    Lt = Ltoa.multiply(((Toz)).multiply((ee.Image(1).divide(cosdSunZe)).add(ee.Image(1).divide(cosdSatZe))).exp())

    # Rayleigh optical thickness
    Tr = (P.divide(Po)).multiply(ee.Image(0.008569).multiply(bandCenter.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter.pow(-4)))))

    # Specular reflection (s- and p- polarization states)
    theta_V = ee.Image(0.0000000001)
    sin_theta_j = sindSunZe.divide(ee.Image(1.333))

    theta_j = sin_theta_j.asin().multiply(ee.Image(180).divide(pi))
    theta_SZ = SunZe

    R_theta_SZ_s = (((theta_SZ.multiply(pi.divide(ee.Image(180)))).subtract(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ.multiply(pi.divide(ee.Image(180)))).add(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)))

    R_theta_V_s = ee.Image(0.0000000001)

    R_theta_SZ_p = (((theta_SZ.multiply(pi.divide(180))).subtract(theta_j.multiply(pi.divide(180)))).tan().pow(2)).divide((((theta_SZ.multiply(pi.divide(180))).add(theta_j.multiply(pi.divide(180)))).tan().pow(2)))

    R_theta_V_p = ee.Image(0.0000000001)

    R_theta_SZ = ee.Image(0.5).multiply(R_theta_SZ_s.add(R_theta_SZ_p))

    R_theta_V = ee.Image(0.5).multiply(R_theta_V_s.add(R_theta_V_p))
  
    # Sun-sensor geometry
    theta_neg = ((cosdSunZe.multiply(ee.Image(-1))).multiply(cosdSatZe)).subtract((sindSunZe).multiply(sindSatZe).multiply(cosdRelAz))

    theta_neg_inv = theta_neg.acos().multiply(ee.Image(180).divide(pi))

    theta_pos = (cosdSunZe.multiply(cosdSatZe)).subtract(sindSunZe.multiply(sindSatZe).multiply(cosdRelAz))

    theta_pos_inv = theta_pos.acos().multiply(ee.Image(180).divide(pi))

    cosd_tni = theta_neg_inv.multiply(pi.divide(180)).cos(); # in degrees

    cosd_tpi = theta_pos_inv.multiply(pi.divide(180)).cos(); # in degrees

    Pr_neg = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni.pow(2))))

    Pr_pos = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi.pow(2))))
  
    # Rayleigh scattering phase function
    Pr = Pr_neg.add((R_theta_SZ.add(R_theta_V)).multiply(Pr_pos))

    # rayleigh radiance contribution
    denom = ee.Image(4).multiply(pi).multiply(cosdSatZe)
    Lr = (ESUN.multiply(Tr)).multiply(Pr.divide(denom))

    # rayleigh corrected radiance
    Lrc = Lt.subtract(Lr)
    LrcImg = Lrc.arrayProject([0]).arrayFlatten([bands])
    prcImg = (Lrc.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)))
    prcImg = prcImg.arrayProject([0]).arrayFlatten([bands])

    # Aerosol Correction

    # Bands in nm
    bands_nm = ee.Image(442).addBands(ee.Image(492)).addBands(ee.Image(559)).addBands(ee.Image(665)).addBands(ee.Image(703)).addBands(ee.Image(739)).addBands(ee.Image(779)).addBands(ee.Image(833)).addBands(ee.Image(864))                            .addBands(ee.Image(0)) .addBands(ee.Image(0)).toArray().toArray(1)

    # Lam in SWIR bands
    Lam_10 = LrcImg.select('B11') # = 0
    Lam_11 = LrcImg.select('B12') # = 0 

    # Calculate aerosol type
    eps = ((((Lam_11).divide(ESUNImg.select('B12'))).log()).subtract(((Lam_10).divide(ESUNImg.select('B11'))).log())).divide(ee.Image(2190).subtract(ee.Image(1610)))

    # Calculate multiple scattering of aerosols for each band
    Lam = (Lam_11).multiply(((ESUN).divide(ESUNImg.select('B12')))).multiply((eps.multiply(ee.Image(-1))).multiply((bands_nm.divide(ee.Image(2190)))).exp())

    # diffuse transmittance
    trans = Tr.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe)).exp();

    # Compute water-leaving radiance
    Lw = Lrc.subtract(Lam).divide(trans);

    # water-leaving reflectance 
    pw = (Lw.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));

    # remote sensing reflectance
    Rrs_S2B = (pw.divide(pi).arrayProject([0]).arrayFlatten([bands]).slice(0,9));

    # set negatives to null
    Rrs_S2B = Rrs_S2B.updateMask(Rrs_S2B.select('B1').gt(0)).multiply(mask)

    return Rrs.set('system:time_start',img.get('system:time_start'))



In [ ]:
def MAIN_S2A(img):

    # msi bands 
    bands = ['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'];

    # rescale
    rescale = img.select(bands).divide(10000)

    # tile footprint
    footprint = rescale.geometry()

    # dem
    DEM = ee.Image('USGS/SRTMGL1_003').clip(footprint); 

    # ozone
    DU = ee.Image(300);
    #ee.Image(ozone.filterDate(startDate,endDate).filterBounds(footprint).mean());

    # Julian Day
    imgDate = ee.Date(img.get('system:time_start'));
    FOY = ee.Date.fromYMD(imgDate.get('year'),1,1);
    JD = imgDate.difference(FOY,'day').int().add(1);

    # earth-sun distance
    myCos = ((ee.Image(0.0172).multiply(ee.Image(JD).subtract(ee.Image(2)))).cos()).pow(2)
    cosd = myCos.multiply(pi.divide(ee.Image(180))).cos();
    d = ee.Image(1).subtract(ee.Image(0.01673)).multiply(cosd).clip(footprint)

    # sun azimuth
    SunAz = ee.Image.constant(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).clip(footprint);

    # sun zenith
    SunZe = ee.Image.constant(img.get('MEAN_SOLAR_ZENITH_ANGLE')).clip(footprint);
    cosdSunZe = SunZe.multiply(pi.divide(ee.Image(180))).cos(); # in degrees
    sindSunZe = SunZe.multiply(pi.divide(ee.Image(180))).sin(); # in degrees

    # sat zenith
    SatZe = ee.Image.constant(img.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5')).clip(footprint);
    cosdSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).cos();
    sindSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).sin();

    # sat azimuth
    SatAz = ee.Image.constant(img.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5')).clip(footprint);

    # relative azimuth
    RelAz = SatAz.subtract(SunAz);
    cosdRelAz = RelAz.multiply(pi.divide(ee.Image(180))).cos();

    # Pressure
    P = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM)).pow(5.25588)).multiply(0.01)
    Po = ee.Image(1013.25);

    # esun
    ESUN = ee.Image(ee.Array([ee.Image(img.get('SOLAR_IRRADIANCE_B1')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B2')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B3')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B4')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B5')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B6')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B7')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8A')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B11')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B12'))]
                      )).toArray().toArray(1);

    ESUN = ESUN.multiply(ee.Image(1))

    ESUNImg = ESUN.arrayProject([0]).arrayFlatten([bands]);

    # create empty array for the images
    imgArr = rescale.select(bands).toArray().toArray(1);

    # pTOA to Ltoa
    Ltoa = imgArr.multiply(ESUN).multiply(cosdSunZe).divide(pi.multiply(d.pow(2)));

    # band centers
    bandCenter = ee.Image(444).divide(1000).addBands(ee.Image(496).divide(1000)).addBands(ee.Image(560).divide(1000)).addBands(ee.Image(664).divide(1000)).addBands(ee.Image(704).divide(1000)).addBands(ee.Image(740).divide(1000)).addBands(ee.Image(782).divide(1000)).addBands(ee.Image(835).divide(1000)).addBands(ee.Image(865).divide(1000)).addBands(ee.Image(1613).divide(1000)).addBands(ee.Image(2202).divide(1000)).toArray().toArray(1);

    # ozone coefficients
    koz = ee.Image(0.0040).addBands(ee.Image(0.0244)).addBands(ee.Image(0.1052)).addBands(ee.Image(0.0516)).addBands(ee.Image(0.0208)).addBands(ee.Image(0.0112)).addBands(ee.Image(0.0079)).addBands(ee.Image(0.0021)).addBands(ee.Image(0.0019))                          .addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1);

    # Calculate ozone optical thickness
    Toz = koz.multiply(DU).divide(ee.Image(1000));

    # Calculate TOA radiance in the absense of ozone
    Lt = Ltoa.multiply(((Toz)).multiply((ee.Image(1).divide(cosdSunZe)).add(ee.Image(1).divide(cosdSatZe))).exp());

    # Rayleigh optical thickness
    Tr = (P.divide(Po)).multiply(ee.Image(0.008569).multiply(bandCenter.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter.pow(-4)))));

    # Specular reflection (s- and p- polarization states)
    theta_V = ee.Image(0.0000000001);
    sin_theta_j = sindSunZe.divide(ee.Image(1.333));

    theta_j = sin_theta_j.asin().multiply(ee.Image(180).divide(pi));

    theta_SZ = SunZe;

    R_theta_SZ_s = (((theta_SZ.multiply(pi.divide(ee.Image(180)))).subtract(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ.multiply(pi.divide(ee.Image(180)))).add(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)));

    R_theta_V_s = ee.Image(0.0000000001);

    R_theta_SZ_p = (((theta_SZ.multiply(pi.divide(180))).subtract(theta_j.multiply(pi.divide(180)))).tan().pow(2)).divide((((theta_SZ.multiply(pi.divide(180))).add(theta_j.multiply(pi.divide(180)))).tan().pow(2)));

    R_theta_V_p = ee.Image(0.0000000001);

    R_theta_SZ = ee.Image(0.5).multiply(R_theta_SZ_s.add(R_theta_SZ_p));

    R_theta_V = ee.Image(0.5).multiply(R_theta_V_s.add(R_theta_V_p));

    # Sun-sensor geometry
    theta_neg = ((cosdSunZe.multiply(ee.Image(-1))).multiply(cosdSatZe)).subtract((sindSunZe).multiply(sindSatZe).multiply(cosdRelAz));

    theta_neg_inv = theta_neg.acos().multiply(ee.Image(180).divide(pi));

    theta_pos = (cosdSunZe.multiply(cosdSatZe)).subtract(sindSunZe.multiply(sindSatZe).multiply(cosdRelAz));

    theta_pos_inv = theta_pos.acos().multiply(ee.Image(180).divide(pi));

    cosd_tni = theta_neg_inv.multiply(pi.divide(180)).cos(); # in degrees

    cosd_tpi = theta_pos_inv.multiply(pi.divide(180)).cos(); # in degrees

    Pr_neg = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni.pow(2))));

    Pr_pos = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi.pow(2))));

    # Rayleigh scattering phase function
    Pr = Pr_neg.add((R_theta_SZ.add(R_theta_V)).multiply(Pr_pos));

    # rayleigh radiance contribution
    denom = ee.Image(4).multiply(pi).multiply(cosdSatZe);
    Lr = (ESUN.multiply(Tr)).multiply(Pr.divide(denom));

    # rayleigh corrected radiance
    Lrc = Lt.subtract(Lr);
    LrcImg = Lrc.arrayProject([0]).arrayFlatten([bands]);
    prcImg = (Lrc.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));
    prcImg = prcImg.arrayProject([0]).arrayFlatten([bands]);


    # Aerosol Correction 

    # Bands in nm
    bands_nm = ee.Image(444).addBands(ee.Image(496)).addBands(ee.Image(560)).addBands(ee.Image(664)).addBands(ee.Image(703)).addBands(ee.Image(740)).addBands(ee.Image(782)).addBands(ee.Image(835)).addBands(ee.Image(865))                            .addBands(ee.Image(0)) .addBands(ee.Image(0)).toArray().toArray(1);

    # Lam in SWIR bands
    Lam_10 = LrcImg.select('B11')
    Lam_11 = LrcImg.select('B12')

    # Calculate aerosol type
    eps = ((((Lam_11).divide(ESUNImg.select('B12'))).log()).subtract(((Lam_10).divide(ESUNImg.select('B11'))).log())).divide(ee.Image(2190).subtract(ee.Image(1610)));

    # Calculate multiple scattering of aerosols for each band
    Lam = (Lam_11).multiply(((ESUN).divide(ESUNImg.select('B12')))).multiply((eps.multiply(ee.Image(-1))).multiply((bands_nm.divide(ee.Image(2190)))).exp());

    # diffuse transmittance
    trans = Tr.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe)).exp();

    # Compute water-leaving radiance
    Lw = Lrc.subtract(Lam).divide(trans);

    # water-leaving reflectance
    pw = (Lw.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));

    # remote sensing reflectance
    Rrs_S2A = (pw.divide(pi).arrayProject([0]).arrayFlatten([bands]).slice(0,9));

    # set negatives to null
    Rrs_S2A = Rrs_S2A.updateMask(Rrs_S2A.select('B1').gt(0)).multiply(mask)

    return Rrs_S2A.set('system:time_start',img.get('system:time_start'))



In [ ]:
# MAIN for S2B
def MAIN_S2B(img):

    # msi bands 
    bands = ['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'];

    # rescale
    rescale = img.select(bands).divide(10000)

    # tile footprint
    footprint = rescale.geometry()

    # dem
    DEM = ee.Image('USGS/SRTMGL1_003').clip(footprint); 

    # ozone
    DU = ee.Image(ozone.filterDate(startDate,endDate).filterBounds(footprint).mean());

    # Julian Day
    imgDate = ee.Date(img.get('system:time_start'));
    FOY = ee.Date.fromYMD(imgDate.get('year'),1,1);
    JD = imgDate.difference(FOY,'day').int().add(1);

    # earth-sun distance
    myCos = ((ee.Image(0.0172).multiply(ee.Image(JD).subtract(ee.Image(2)))).cos()).pow(2)
    cosd = myCos.multiply(pi.divide(ee.Image(180))).cos();
    d = ee.Image(1).subtract(ee.Image(0.01673)).multiply(cosd).clip(footprint)

    # sun azimuth
    SunAz = ee.Image.constant(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).clip(footprint);

    # sun zenith
    SunZe = ee.Image.constant(img.get('MEAN_SOLAR_ZENITH_ANGLE')).clip(footprint);
    cosdSunZe = SunZe.multiply(pi.divide(ee.Image(180))).cos(); # in degrees
    sindSunZe = SunZe.multiply(pi.divide(ee.Image(180))).sin(); # in degrees

    # sat zenith
    SatZe = ee.Image.constant(img.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5')).clip(footprint);
    cosdSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).cos();
    sindSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).sin();

    # sat azimuth
    SatAz = ee.Image.constant(img.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5')).clip(footprint);

    # relative azimuth
    RelAz = SatAz.subtract(SunAz);
    cosdRelAz = RelAz.multiply(pi.divide(ee.Image(180))).cos();

    # Pressure
    P = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM)).pow(5.25588)).multiply(0.01)
    Po = ee.Image(1013.25);

    # esun
    ESUN = ee.Image(ee.Array([ee.Image(img.get('SOLAR_IRRADIANCE_B1')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B2')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B3')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B4')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B5')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B6')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B7')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8A')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B11')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B12'))]
                      )).toArray().toArray(1);

    ESUN = ESUN.multiply(ee.Image(1))

    ESUNImg = ESUN.arrayProject([0]).arrayFlatten([bands]);

    # create empty array for the images
    imgArr = rescale.select(bands).toArray().toArray(1);

    # pTOA to Ltoa
    Ltoa = imgArr.multiply(ESUN).multiply(cosdSunZe).divide(pi.multiply(d.pow(2)));

    # band centers
    bandCenter = ee.Image(442).divide(1000).addBands(ee.Image(492).divide(1000)).addBands(ee.Image(559).divide(1000)).addBands(ee.Image(665).divide(1000)).addBands(ee.Image(703).divide(1000)).addBands(ee.Image(739).divide(1000)).addBands(ee.Image(779).divide(1000)).addBands(ee.Image(833).divide(1000)).addBands(ee.Image(864).divide(1000)).addBands(ee.Image(1610).divide(1000)).addBands(ee.Image(2185).divide(1000)).toArray().toArray(1);

    # ozone coefficients
    koz = ee.Image(0.0037).addBands(ee.Image(0.0223)).addBands(ee.Image(0.1027)).addBands(ee.Image(0.0505)).addBands(ee.Image(0.0212)).addBands(ee.Image(0.0112)).addBands(ee.Image(0.0085)).addBands(ee.Image(0.0022)).addBands(ee.Image(0.0021))                          .addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1);
    
    # Calculate ozone optical thickness
    Toz = koz.multiply(DU).divide(ee.Image(1000));

    # Calculate TOA radiance in the absense of ozone
    Lt = Ltoa.multiply(((Toz)).multiply((ee.Image(1).divide(cosdSunZe)).add(ee.Image(1).divide(cosdSatZe))).exp());

    # Rayleigh optical thickness
    Tr = (P.divide(Po)).multiply(ee.Image(0.008569).multiply(bandCenter.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter.pow(-4)))));

    # Specular reflection (s- and p- polarization states)
    theta_V = ee.Image(0.0000000001);
    sin_theta_j = sindSunZe.divide(ee.Image(1.333));

    theta_j = sin_theta_j.asin().multiply(ee.Image(180).divide(pi));

    theta_SZ = SunZe;

    R_theta_SZ_s = (((theta_SZ.multiply(pi.divide(ee.Image(180)))).subtract(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ.multiply(pi.divide(ee.Image(180)))).add(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)));

    R_theta_V_s = ee.Image(0.0000000001);

    R_theta_SZ_p = (((theta_SZ.multiply(pi.divide(180))).subtract(theta_j.multiply(pi.divide(180)))).tan().pow(2)).divide((((theta_SZ.multiply(pi.divide(180))).add(theta_j.multiply(pi.divide(180)))).tan().pow(2)));

    R_theta_V_p = ee.Image(0.0000000001);

    R_theta_SZ = ee.Image(0.5).multiply(R_theta_SZ_s.add(R_theta_SZ_p));

    R_theta_V = ee.Image(0.5).multiply(R_theta_V_s.add(R_theta_V_p));

    # Sun-sensor geometry
    theta_neg = ((cosdSunZe.multiply(ee.Image(-1))).multiply(cosdSatZe)).subtract((sindSunZe).multiply(sindSatZe).multiply(cosdRelAz));

    theta_neg_inv = theta_neg.acos().multiply(ee.Image(180).divide(pi));

    theta_pos = (cosdSunZe.multiply(cosdSatZe)).subtract(sindSunZe.multiply(sindSatZe).multiply(cosdRelAz));

    theta_pos_inv = theta_pos.acos().multiply(ee.Image(180).divide(pi));

    cosd_tni = theta_neg_inv.multiply(pi.divide(180)).cos(); # in degrees

    cosd_tpi = theta_pos_inv.multiply(pi.divide(180)).cos(); # in degrees

    Pr_neg = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni.pow(2))));

    Pr_pos = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi.pow(2))));

    # Rayleigh scattering phase function
    Pr = Pr_neg.add((R_theta_SZ.add(R_theta_V)).multiply(Pr_pos));

    # rayleigh radiance contribution
    denom = ee.Image(4).multiply(pi).multiply(cosdSatZe);
    Lr = (ESUN.multiply(Tr)).multiply(Pr.divide(denom));

    # rayleigh corrected radiance
    Lrc = Lt.subtract(Lr);
    LrcImg = Lrc.arrayProject([0]).arrayFlatten([bands]);
    prcImg = (Lrc.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));
    prcImg = prcImg.arrayProject([0]).arrayFlatten([bands]);

    # Aerosol Correction 

    # Bands in nm
    bands_nm = ee.Image(442).addBands(ee.Image(492)).addBands(ee.Image(559)).addBands(ee.Image(665)).addBands(ee.Image(703)).addBands(ee.Image(739)).addBands(ee.Image(779)).addBands(ee.Image(833)).addBands(ee.Image(864))                            .addBands(ee.Image(0)) .addBands(ee.Image(0)).toArray().toArray(1);

    # Lam in SWIR bands
    Lam_10 = LrcImg.select('B11'); # = 0
    Lam_11 = LrcImg.select('B12'); # = 0 

    # Calculate aerosol type
    eps = ((((Lam_11).divide(ESUNImg.select('B12'))).log()).subtract(((Lam_10).divide(ESUNImg.select('B11'))).log())).divide(ee.Image(2190).subtract(ee.Image(1610)));

    # Calculate multiple scattering of aerosols for each band
    Lam = (Lam_11).multiply(((ESUN).divide(ESUNImg.select('B12')))).multiply((eps.multiply(ee.Image(-1))).multiply((bands_nm.divide(ee.Image(2190)))).exp());

    # diffuse transmittance
    trans = Tr.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe)).exp();

    # Compute water-leaving radiance
    Lw = Lrc.subtract(Lam).divide(trans);

    # water-leaving reflectance 
    pw = (Lw.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));

    # remote sensing reflectance
    Rrs_S2B = (pw.divide(pi).arrayProject([0]).arrayFlatten([bands]).slice(0,9));

    # set negatives to null
    Rrs_S2B = Rrs_S2B.updateMask(Rrs_S2B.select('B1').gt(0)).multiply(mask)

    return Rrs_S2B.set('system:time_start',img.get('system:time_start'))



In [ ]:
# Collection Processing 

# atmospheric correction for S2A
Rrs_S2A = FC_S2A.map(MAIN_S2A)

# atmopsheric correction for S2B
Rrs_S2B = FC_S2B.map(MAIN_S2B)

# Merge S2A + S2B collection
Rrs_AB_combined = Rrs_S2A.merge(Rrs_S2B).sort('system:time_start')

# Merged List
combinedList = Rrs_AB_combined.toList(1000000)
target_rrs_image = ee.Image(combinedList.get(target_image_number))

In [ ]:
Map = folium.Map()
Map.setOptions()
lat = 33.37109359117551
lon = -97.06406547865038
Viz = {'min': 0, 'max': 0.015, 'palette': ['darkblue', 'blue', 'cyan', 'limegreen', 'yellow', 'orange', 'orangered', 'red', 'darkred']}

Map.setCenter(lon, lat, 10) 
outputGeometry = target_rrs_image.select('B2')
Map.addLayer(outputGeometry, Viz)

In [ ]:
Map